In [1]:
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
!nvidia-smi

Wed Jul 14 15:38:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datetime import datetime
import matplotlib.pyplot as plt
X = np.load('/content/drive/MyDrive/XbrainRegions.npy')
y = np.load('/content/drive/MyDrive/YbrainRegions.npy')

X.shape

(678, 360)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(f'X_train shape is {X_train.shape}\nX_test shape is {X_test.shape} \ny_train shape is {y_train.shape} \ny_test shape is {y_test.shape}')
scaler_X = StandardScaler()
f_transformer = scaler_X.fit(X_train)

X_train = f_transformer.transform(X_train)
X_test = f_transformer.transform(X_test)

X_train shape is (508, 360)
X_test shape is (170, 360) 
y_train shape is (508,) 
y_test shape is (170,)


In [7]:
torch.Tensor(X_train[0][:5])

tensor([-0.1422, -1.7029,  0.7862, -0.2367, -0.6796])

In [8]:
class BrainData(Dataset):
  def __init__(self, X , y):
    self.Xdata = X
    self.ydata = y

  def __len__(self):
    return len(self.Xdata)

  def __getitem__(self , idx):
    oneXdata = self.Xdata[idx]
    oneydata = self.ydata[idx]

    return {'Xdata':torch.as_tensor(oneXdata) , 'ydata':torch.as_tensor(oneydata , dtype=torch.long) }


In [9]:
trainData = BrainData(X_train , y_train)
testData = BrainData(X_test , y_test)
batch_size = 16
trainDataLoader = DataLoader(trainData , batch_size=batch_size)
testDataLoader = DataLoader(testData , batch_size=batch_size)

In [10]:
for i in trainDataLoader:
  print(i)
  break

{'Xdata': tensor([[-0.1422, -1.7029,  0.7862,  ...,  0.3148,  1.4860,  0.5216],
        [ 1.3487,  0.6773,  0.3015,  ...,  0.5750,  0.4483,  0.1074],
        [ 0.8327,  0.6699,  0.4024,  ...,  0.7074, -0.1462, -0.4517],
        ...,
        [ 0.3092, -0.4280,  0.9820,  ...,  0.2023,  1.0154, -0.0419],
        [ 0.4603,  0.3684,  0.0385,  ..., -0.7210, -0.3252,  0.3339],
        [ 0.1625, -0.8549, -1.2531,  ..., -0.1456,  0.9415, -3.1991]],
       dtype=torch.float64), 'ydata': tensor([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0])}


In [101]:
class BinaryPrediction(nn.Module):
  def __init__(self, numofHiddenNeurons):
    super().__init__()
    self.out1 = nn.Linear(360 ,numofHiddenNeurons )
    self.out2 = nn.Linear(numofHiddenNeurons , 256)
    self.out3 = nn.Linear(256 , 2)
    self.nonLinearity = nn.ReLU()
    self.dropout = nn.Dropout(0.1)
  

  def forward(self , X):
    o = self.out1(X)
    o = self.dropout(o)
    o = self.nonLinearity(o)
    o = self.out2(o)
    o = self.dropout(o)
    o = self.nonLinearity(o)
    o = self.out3(o)

    return o


model = BinaryPrediction(512)
model = model.to(device)




In [102]:
EPOCHS = 50
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss().to(device)

In [103]:
model.double()

BinaryPrediction(
  (out1): Linear(in_features=360, out_features=512, bias=True)
  (out2): Linear(in_features=512, out_features=256, bias=True)
  (out3): Linear(in_features=256, out_features=2, bias=True)
  (nonLinearity): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [104]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device,  
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    X = d["Xdata"].to(device)
    targets = d["ydata"].to(device)

    outputs = model(X)

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  

  return correct_predictions.double() / n_examples, np.mean(losses)

In [105]:
float(len(X_train))

508.0

In [106]:

for epoch in range(EPOCHS):
  
  
  train_acc, train_loss = train_epoch(
    model,
    trainDataLoader,    
    loss_fn, 
    optimizer, 
    device, 
    float(len(X_train))
  )
  if((epoch+1) %10 == 0):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

  
    print(f'Train loss {train_loss} accuracy {train_acc}')


  



Epoch 10/50
----------
Train loss 0.6014170930743316 accuracy 0.7519685039370079
Epoch 20/50
----------
Train loss 0.48835581646444703 accuracy 0.860236220472441
Epoch 30/50
----------
Train loss 0.3711421143672693 accuracy 0.9035433070866141
Epoch 40/50
----------
Train loss 0.2818467871910084 accuracy 0.9251968503937008
Epoch 50/50
----------
Train loss 0.20612590217763127 accuracy 0.9665354330708661


In [107]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
    
      X = d["Xdata"].to(device)
      targets = d["ydata"].to(device)

      outputs = model(X)


      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [108]:
test_acc, _ = eval_model(
  model,
  testDataLoader ,
  loss_fn,
  device,
  len(X_test)
)

test_acc

tensor(0.8882, device='cuda:0', dtype=torch.float64)